In [ ]:
def exponential_mechanism_position(corr_df, elbow_index, epsilon):
    d = len(corr_df)
    K = d // 5

    indices = np.arange(len(corr_df))
    mask = np.abs(indices - elbow_index) <= K
    candidate_indices = indices[mask]

    utilities = -np.abs(candidate_indices - elbow_index)
    
    # Exponential mechanism probabilities
    scores = np.exp((epsilon * utilities) / 2)
    probabilities = scores / scores.sum()

    sampled_index = np.random.choice(candidate_indices, p=probabilities)

    return sampled_index, probabilities, candidate_indices

In [ ]:
corr_df = corr_df.reset_index(drop=True)
corr_df['diffrence'] = corr_df['Spearman_Correlation'].diff(1)
corr_df['change'] = corr_df['diffrence'].diff(1)
print(corr_df['change'])
print(np.argmin(corr_df['change']))

CHANGE İÇİN CONSTRAİNLİ EN YÜKSEĞİ EL baştan %10 sondan %50 sonra pozisyon bazlı Exponential Mechanishm uygula. DP sağlar.

Applying the mechanism

In [ ]:
epsilon = 1.0 # privacy budget for elbow selection

dp_elbow, probs, candidate_indices = exponential_mechanism_position( corr_df, elbow_index, epsilon )

print("DP elbow index:", dp_elbow)
print("Selected feature:", corr_df.loc[dp_elbow, "Feature"])

probability mass

In [ ]:
prob_df = corr_df.copy()
prob_df["probability"] = 0.0 # initialize all to zero
prob_df.iloc[candidate_indices, prob_df.columns.get_loc("probability")] = probs
prob_df["distance_to_elbow"] = abs(prob_df.index - elbow_index)

prob_df.sort_values("probability", ascending=False)